#### Set-up for Colab
Install dependencies, mount drive, etc.

In [ ]:
#pip install -q -U tensorflow-text

In [ ]:
#pip install -q tf-models-official==2.4.0

In [ ]:
pip uninstall tensorflow

In [ ]:
pip install tensorflow==2.5.0

     |████████████████████████████████| 454.3 MB 18 kB/s 
     |████████████████████████████████| 1.2 MB 44.4 MB/s 
     |████████████████████████████████| 4.0 MB 43.1 MB/s 
     |████████████████████████████████| 462 kB 57.7 MB/s 
  Created wheel for wrapt: filename=wrapt-1.12.1-cp37-cp37m-linux_x86_64.whl size=68725 sha256=973081f200384c89faf3eede8b3f25b00ad4361c234f4ca359d92a8a39af2803
  Stored in directory: /root/.cache/pip/wheels/62/76/4c/aa25851149f3f6d9785f6c869387ad82b3fd37582fa8147ac6
Successfully built wrapt
  Attempting uninstall: typing-extensions
    Found existing installation: typing-extensions 3.10.0.2
    Uninstalling typing-extensions-3.10.0.2:
      Successfully uninstalled typing-extensions-3.10.0.2
  Attempting uninstall: grpcio
    Found existing installation: grpcio 1.42.0
    Uninstalling grpcio-1.42.0:
      Successfully uninstalled grpcio-1.42.0
  Attempting uninstall: wrapt
    Found existing installation: wrapt 1.13.3
    Uninstalling wrapt-1.13.3:
      Suc

In [ ]:
pip install transformers

     |████████████████████████████████| 3.1 MB 5.4 MB/s 
     |████████████████████████████████| 596 kB 47.8 MB/s 
     |████████████████████████████████| 59 kB 4.2 MB/s 
     |████████████████████████████████| 3.3 MB 16.6 MB/s 
     |████████████████████████████████| 895 kB 46.9 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
pip install bert-for-tf2

     |████████████████████████████████| 41 kB 135 kB/s 
  Created wheel for bert-for-tf2: filename=bert_for_tf2-0.14.9-py3-none-any.whl size=30534 sha256=c1de66dd7b2185cb221b4aadf918f64aa1353763754f04086f36124e9f2659d6
  Stored in directory: /root/.cache/pip/wheels/47/b6/e5/8c76ec779f54bc5c2f1b57d2200bb9c77616da83873e8acb53
  Created wheel for params-flow: filename=params_flow-0.8.2-py3-none-any.whl size=19473 sha256=b1e9b644c8ae0587dea0ae0b99ec8c25134e3483bbfe51f308a8806518a8f8cb
  Stored in directory: /root/.cache/pip/wheels/0e/fc/d2/a44fff33af0f233d7def6e7de413006d57c10e10ad736fe8f5
  Created wheel for py-params: filename=py_params-0.10.2-py3-none-any.whl size=7912 sha256=c1d80eb2e05939b5dd4fdfd0adde3643597549229c6f612cf32bda629837535b
  Stored in directory: /root/.cache/pip/wheels/e1/11/67/33cc51bbee127cb8fb2ba549cd29109b2f22da43ddf9969716
Successfully built bert-for-tf2 params-flow py-params


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
path = 'drive/MyDrive/MIDS/chemical_patent_cer_ee'
# path = '..'

#### Import Libraries

In [ ]:
import os
import io
import re
import sys
import sys
sys.path.append('/content/drive/MyDrive/MIDS/chemical_patent_cer_ee/notebooks')

import numpy as np
import pandas as pd
import argparse
from time import time
import matplotlib.pyplot as plt

import pickle
from csv import reader

import tensorflow as tf
from transformers import BertTokenizer
import bert

from sre_inputs import *
from train_test import *
from sre_models import *

#### BERT Model
- Load BERT model and tokenizer
- Set max length for inputs

In [ ]:
# path for bert model
bert_model_dir = f'{path}/bert/bert_mini'
bert_type = bert_model_dir.split('/')[-1]

# set tokenizer
vocab_file = os.path.join(bert_model_dir, "vocab.txt")
tokenizer = BertTokenizer(vocab_file=vocab_file, do_lower_case=False)

# set BERT model
bert_params = bert.params_from_pretrained_ckpt(bert_model_dir)
bert = bert.BertModelLayer.from_params(bert_params, name="bert")

# set max length for inputs
max_length = 512

# set parameters for model type
marker_type = 'em' # 'em', 'ner', or 'std'
head_type = 'start' # 'cls', 'start', 'pool', or 'ner'
subsampled = False

#### Data
- Upload preprocessed chemical patent file(s)
- Use `sre_inputs` module to generate inputs for model
- Sample only: split into train/test using `train_test` module
- Need to one hot encode labels before using in model

*NB: Make sure that preprocessed data being uploaded and parameters chosen for generating inputs **both** match the type of model it will be used for!*

In [ ]:
#### TRAIN/DEV DATASET PROCESSING ####

# paths for preprocessed data
if (marker_type == 'em' or marker_type == 'std') and not subsampled:
    train_path = f'{path}/data/sre_em/sre_em_train.csv'
    dev_path = f'{path}/data/sre_em/sre_em_dev.csv'
elif (marker_type == 'em' or marker_type == 'std') and subsampled:
    train_path = f'{path}/data/sre_em/sre_em_train_subsampled.csv'
    dev_path = f'{path}/data/sre_em/sre_em_dev_subsampled.csv'
elif marker_type == 'ner' and not subsampled:
    train_path = f'{path}/data/sre_ner/sre_ner_train.csv'
    dev_path = f'{path}/data/sre_ner/sre_ner_dev.csv'
elif marker_type == 'ner' and subsampled:
    train_path = f'{path}/data/sre_ner/sre_ner_train_subsampled.csv'
    dev_path = f'{path}/data/sre_ner/sre_ner_dev_subsampled.csv'

print(f'Loaded {train_path}')
print(f'Loaded {dev_path}')

# generate inputs for model
if marker_type == 'em' or marker_type == 'ner':
    train_lists = generate_entity_inputs(train_path, tokenizer, marker_type, head_type, max_length)
    dev_lists = generate_entity_inputs(dev_path, tokenizer, marker_type, head_type, max_length)
elif marker_type == 'std':
    train_lists = generate_standard_inputs(train_path, tokenizer, max_length)
    dev_lists = generate_standard_inputs(dev_path, tokenizer, max_length)

# generate inputs and labels
# one hot encode labels
model_inputs_train = [x for x in train_lists[0][:5]]
train_labels = train_lists[1]
model_labels_train = tf.one_hot(train_labels, depth=3)

model_inputs_dev = [x for x in dev_lists[0][:5]]
dev_labels = dev_lists[1]
model_labels_dev = tf.one_hot(dev_labels, depth=3)

Loaded drive/MyDrive/MIDS/chemical_patent_cer_ee/data/sre_em/sre_em_train.csv
Loaded drive/MyDrive/MIDS/chemical_patent_cer_ee/data/sre_em/sre_em_dev.csv


In [ ]:
#### TEST DATASET PROCESSING ####

# path for preprocessed data
if marker_type == 'em' or marker_type == 'std':
    test_path = f'{path}/data/sre_em/sre_em_test.csv'
elif marker_type == 'ner':
    test_path = f'{path}/data/sre_ner/sre_ner_test.csv'

# generate inputs for model
if marker_type == 'em' or marker_type == 'ner':
    test_lists = generate_entity_inputs(test_path, tokenizer, marker_type, head_type, max_length)
elif marker_type == 'std':
    test_lists = generate_standard_inputs(test_path, tokenizer, max_length)

# generate inputs and labels
# one hot encode labels
model_inputs_test = [x for x in test_lists[0][:5]]
test_labels = test_lists[1]
model_labels_test = tf.one_hot(test_labels, depth=3)

#### Run Model(s)

In [ ]:
#### TRAIN/DEV RUN ####

tf.keras.backend.clear_session()
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)

epochs = 10
batch_size = 32
train_layers = 0

if head_type == 'cls':
    model = sre_cls_model(bert, max_length, train_layers)
elif head_type == 'start':
    model = sre_start_model(bert, max_length, train_layers)
elif head_type == 'pool':
    model = sre_pool_model(bert, max_length, train_layers)
elif head_type == 'ner':
    model = sre_pool_model(bert, max_length, train_layers)

if head_type == 'cls':
    model.fit(
        model_inputs_train[:3], 
        {"sre": model_labels_train},
        validation_data=(model_inputs_dev[:3], {"sre": model_labels_dev}),
        epochs=epochs,
        batch_size=batch_size
    )
else:
    model.fit(
    model_inputs_train, 
    {"sre": model_labels_train},
    validation_data=(model_inputs_dev, {"sre": model_labels_dev}),
    epochs=epochs,
    batch_size=batch_size
)

In [ ]:
# evaluate model on test data
print('Evaluate on test data')

if marker_type == 'std' or head_type == 'cls':
    results = model.evaluate(model_inputs_test[:3], model_labels_test, batch_size=batch_size)
else:
    results = model.evaluate(model_inputs_test, model_labels_test, batch_size=batch_size)

# generate predictions on new data (probabilities -- the output of the last layer)
print("Generate predictions for new samples")

if marker_type == 'std' or head_type == 'cls':
    predictions = model.predict(model_inputs_test[:3])
else: 
    predictions = model.predict(model_inputs_test)

print("predictions shape:", predictions.shape)

# save stuff
if subsampled:
    model_name = f'SRE_{bert_type}_{marker_type}_{head_type}_sub'
else:
    model_name = f'SRE_{bert_type}_{marker_type}_{head_type}'

# save results and predictions
outputs = [results, predictions]
with open(f'{path}/results/{model_name}.pickle', "wb") as f:
    pickle.dump(outputs, f)

# save model
# model.save(f'{path}/models/{model_name}')

In [ ]:
# # how to open saved file
# with open(f'{path}/results/{model_name}.pickle', "rb") as f:
#     saved_outputs = pickle.load(f)

In [ ]:
# OPTIONAL: visualize model
#tf.keras.utils.plot_model(model, show_shapes=True, dpi=48)